### Summary 
This file can be used to download a training set of high-resolution satellite images within a defined map extent. To begin, ensure that the *api_key* is defined. To generate an API key, follow the instructions on the [Maps Static API Developer page](https://developers.google.com/maps/documentation/maps-static/get-api-key). Then, define a *search_extent* for which high-resolution satellite tiles will be downloaded. 

Search extent may be defined as an extent box in lattitude and longitude pairs. Images are downloaded as 600x600 pixel tiles; while there is no defined stride (no overlap across images), this may be added using a defined extent box buffer. The desired output path may also be defined under *save_path*. 

In [ ]:
from math import cos
from math import pi
import math

In [1]:
def get_bounds(c_lat, c_lng, zoom, w, h):

    res_lat = (cos(c_lat * pi / 180.0) * 360.0) / (256 * 2 ** zoom)
    res_lng = 360.0/(256 * 2 ** zoom)

    return [(c_lat-((res_lat * h) / 2), c_lng-((res_lng * w) / 2)), 
            (c_lat+((res_lng * w) / 2), c_lng+((res_lng * w) / 2))]

#### Download Satellite Images using Extent Area
Using a defined square region, download collection of 600x600 high-resolution satellite imagery tiles. Resolution and image tile size may be changed by modifying defined constants.

In [2]:
def download_area(api_key, search_extent, save_path):
  # search extent format: [min_lat, max_lat, min_long, max_long]

  centerLat = (search_extent[1] + search_extent[0]) / 2
  centerLon = (search_extent[3] + search_extent[2]) / 2
  zoom = 18
  mapWidth = 600
  mapHeight = 600
  strides = get_bounds(centerLat, centerLon, zoom, mapWidth, mapHeight)
  lat_stride = (strides[1][0] - strides[0][0])
  lng_stride = (strides[1][1] - strides[0][1])

  lat_range = search_extent[1] - search_extent[0]
  lng_range = search_extent[3] - search_extent[2]

  # request and downloadd images
  counter = 1

  for i in range(math.ceil(lat_range/lat_stride)):

      for j in range(math.ceil(lng_range/lng_stride)):
          centerLat = search_extent[0] + lat_stride*i
          centerLon =search_extent[2] + lng_stride*j
          zoom = 18
          mapWidth = 600
          mapHeight = 600
          get_bounds(centerLat, centerLon, zoom, mapWidth, mapHeight)

          full_file = ("https://maps.googleapis.com/maps/api/staticmap?center=" 
                    + str(centerLat) 
                    + "," + str(centerLon) + 
                    "&zoom=" + str(zoom) + 
                    "&size=" + str(mapWidth) + "x" 
                    + str(mapHeight) + 
                    "&maptype=satellite&key=" + api_key)

          img_data = requests.get(full_file).content
          with open(save_path + 'sat_tile' + str(counter) + '.jpg', 'wb') as handler:
              handler.write(img_data)

          counter += 1


#### Download Satellite Images using Lat/Long Coordinates
Download square tiles around defined coordinate set parameteer. Can be used to augment OSM data for market location prediction. 

In [3]:
def download_locations(api_key, coord_file, save_path):
  reg = pd.read_csv(coord_file)

  for index, row in reg.iterrows():

    lat = row['lat']
    lon = row['long']
    counter = 1 

    if not math.isnan(lat) and not math.isnan(lon):
      # row has data 

      if row["is_market"] == 1:
        # determined as market from OSM data, no need for satellite augmentation
        augmented_markets.loc[len(augmented_markets)] = [row['region'], lat, lon, 1]
      
      else:
        zoom = 19
        mapWidth = 600
        mapHeight = 600

        full_file = ("https://maps.googleapis.com/maps/api/staticmap?center=" 
                  + str(lat) 
                  + "," + str(lon) + 
                  "&zoom=" + str(zoom) + 
                  "&size=" + str(mapWidth) + "x" 
                  + str(mapHeight) + 
                  "&maptype=satellite&key=" + api_key)

        img_data = requests.get(full_file).content
        with open('sat_tile' + str(counter) + '.jpg', 'wb') as handler:
            handler.write(img_data)
        counter += 1